In [0]:
# Autor: Jairo Cifuentes
# Fecha: 06/08/2025
# Descripción: Carga de datos dn_user en Bronze

In [0]:
#Configuración de conexión JDBC
JDBC_CONFIG = {
    "hostname": "psql-dn-keycloak-restore-2.postgres.database.azure.com",
    "port": 5432,
    "database": "keycloak",
    "username": dbutils.secrets.get(scope='secret-storeview', key='username-keycloak-db'),
    "password": dbutils.secrets.get(scope='secret-storeview', key='password-keycloak-db'),
    "driver": "org.postgresql.Driver"
}

jdbcUrl = f"jdbc:postgresql://{JDBC_CONFIG['hostname']}:{JDBC_CONFIG['port']}/{JDBC_CONFIG['database']}?sslmode=require"
connectionProperties = {
    "user": JDBC_CONFIG["username"],
    "password": JDBC_CONFIG["password"],
    "driver": JDBC_CONFIG["driver"]
}

In [0]:
#Se importa los tipos de datos necesarios para definir el esquema del DataFrame
from pyspark.sql.types import StructType, StructField, StringType, BooleanType

# Controlar los tipos de datos y evitar inferencias automáticas de Spark
schema_dn_user = StructType([
    StructField("parent_group", StringType(), True),
    StructField("grupo", StringType(), True),
    StructField("nombre", StringType(), True),
    StructField("apellido", StringType(), True),
    StructField("email", StringType(), True),
    StructField("enabled", BooleanType(), True),
    StructField("verified", BooleanType(), True)
])

In [0]:
# Leer la tabla "dn_user" desde PostgreSQL usando el esquema definido
# Para garantizar que los tipos de datos sean los correctos
df_dn_user = spark.read \
    .format("jdbc") \
    .option("url", jdbcUrl) \
    .option("dbtable", "dn_user") \
    .option("user", connectionProperties["user"]) \
    .option("password", connectionProperties["password"]) \
    .option("driver", connectionProperties["driver"]) \
    .schema(schema_dn_user) \
    .load()

# Mostrar los primeros 10 registros para verificar la lectura
display(df_dn_user.limit(10))


In [0]:
#Validar el esquema
def validar_esquema_postgresql(jdbc_url, table_name, connection_props, schema_dn_user):

    # Leer la tabla desde PostgreSQL usando el esquema definido
    df_actual = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_props)
    actual_schema = df_actual.schema

    # Comparar los esquemas
    expected_fields = {field.name: type(field.dataType) for field in schema_dn_user.fields}
    actual_fields = {field.name: type(field.dataType) for field in actual_schema.fields}

    errores = False # Variable para controlar si hay errores

    #Revisar si hay colummnas faltantes o con tipos diferentes
    for col in expected_fields:
        if col not in actual_fields:
            print(f"⚠️ Columna faltante: '{col}'")
            errores = True
        elif expected_fields[col] != actual_fields[col]:
            print(f"⚠️ Tipo cambiado en '{col}': esperado {expected_fields[col]}, recibido {actual_fields[col]}")
            errores = True

    #Revisar si hay columnas adicionales
    for col in actual_fields:
        if col not in expected_fields:
            print(f"⚠️ Columna adicional no esperada: '{col}'")
            errores = True

    # Mostrar el resdultado de si hubo o no errores
    if errores:
        print("❌ El esquema actual NO coincide con el esperado.")
    else:
        print("✅ El esquema actual coincide con el esperado.")

        # Mostrar los primeros 10 registros para verificar la lectura
        display(df_actual.limit(10))

# Ejecutar la validación
validar_esquema_postgresql(jdbcUrl, "dn_user", connectionProperties, schema_dn_user)

In [0]:
#Leer objeto desde PostgreSQL (quitar si es necesario ya que ahora se hace con schema)
df_dn_user = spark.read.jdbc(url=jdbcUrl, table="dn_user", properties=connectionProperties)
display(df_dn_user.limit(10))

In [0]:
# Ruta destino en Bronze
bronze_path = "/mnt/bronze/keycloak/dn_user"

# Eliminar en caso que exista
try:
  dbutils.fs.ls(bronze_path)
  print("Existe, se sobreescribirá")
except:
  print("No existe, se creará")

# Escribir en delta
df_dn_user.write.format("delta").mode("overwrite").save(bronze_path)

# Registrar tabla
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze.bronze_dn_user
    USING DELTA
    LOCATION '{bronze_path}/'
""")
spark.sql("REFRESH TABLE bronze.bronze_dn_user")

In [0]:
# Verificar lectura
df_val = spark.table("bronze.bronze_dn_user")
display(df_val.limit(10))